In [70]:
import base64
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os
import pickle
import pandas as pd

Senha incorreta


In [ ]:
def criar_hash_ofuscada(info, passphrase):
    salt = os.urandom(16)

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    chave = kdf.derive(passphrase.encode('utf-8'))

    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(chave), modes.CFB(iv), backend=default_backend())

    # Codificar dados em Base64 antes de criptografar
    senha_base64 = base64.b64encode(info['senha'].encode('utf-8')).decode('utf-8')
    encryptor = cipher.encryptor()
    senha_ofuscada = encryptor.update(senha_base64.encode('utf-8')) + encryptor.finalize()

    return {'ip': info['ip'], 'login': info['login'], 'senha': senha_ofuscada, 'salt': salt, 'iv': iv}

def criar_arquivo_ofuscado(dados, arquivo_ofuscado, passphrase):
    dados_ofuscados = {id: criar_hash_ofuscada(info, passphrase) for id, info in dados.items()}

    with open(arquivo_ofuscado, 'wb') as f_ofuscado:
    pickle.dump(dados_ofuscados, f_ofuscado)

        print(f"Todos os dados ofuscados e salvos em {arquivo_ofuscado}")
def criar_dataframe(dict):
    df_sem_ordem = pd.DataFrame(columns=['IP', 'Login', 'Senha'])
    df = df_sem_ordem.sort_values(by='IP')

    for login,informacoes in dict.items():     
        df = df.append({'Login': login, 'IP': informacoes['IP'] ,'Senha': informacoes['Senha']}, ignore_index=True)

    return df
    

In [ ]:
if __name__ == "__main__":
    dados_senhas = {
        'email1': {'IP': '192.168.0.1', 'Senha': 'senha1'},
        'email2': {'IP': '192.168.0.2', 'Senha': 'senha2'},
        'email3': {'IP': '192.168.0.3', 'Senha': 'senha3'}
    }

    dados_senha_df = criar_dataframe(dados_senhas)
    

    arquivo_ofuscado = "sicdc.bin"
    passphrase = "Cnpa2024#"

    criar_arquivo_ofuscado(dados_senhas_df, arquivo_ofuscado, passphrase)